In [1]:
#!/usr/bin/env python
# coding: utf-8

# This code collects summa parameter samples. 

import os, sys, argparse, datetime, shutil
from glob import glob
import netCDF4 as nc
import numpy as np
import xarray as xr
import pandas as pd
from tqdm import tqdm

# Function to extract a given setting from the configuration file
def read_from_control(control_file, setting):
    
    # Open 'control_active.txt' and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('|',1)[1].split('#',1)[0].strip() 
    # Return this value    
    return substring


# main
if __name__ == '__main__':
    
    # ----------------------------- Settings ------------------------------        
    # inputs
    calib_output_path = '/home/h294liu/project/proj/5_summaCalib/BowAtBanff_LA_calib/GLUE_summary'
    success_runs_txt = os.path.join(calib_output_path, '6_collect_run_records/runs_success.txt')       # text file to save success run info  
    control_file = os.path.join(calib_output_path, '6_collect_run_records/control_active.txt')         # active control file
    paramFile_basename = 'trialParams.nc'
    statsFile_basename = 'trial_stats.txt'
    
    output_path = os.path.join(calib_output_path, '7_collect_param_samples') # output text file to save summa param and obj func 
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # --------------------------- End settings -----------------------------        
    
    # 1. Read interested param name list   
    object_parameters = read_from_control(control_file, 'object_parameters')
    object_parameters = object_parameters.split(',')
    object_parameters = [x.strip() for x in object_parameters]    
    Nparam = len(object_parameters)
    
    # 2. Read path list of successful model runs and ostModel.txt
    success_runs = np.loadtxt(success_runs_txt,dtype='str')
    Nsample = len(success_runs)    

    # 3. Identify GRU and HRU dimension variables 
    iRun = 0
    iRun_path = success_runs[iRun]

    paramFile = os.path.join(iRun_path,paramFile_basename)
    f    = xr.open_dataset(paramFile)
    
    # identify numbers of GRUs and HRUs
    nGru = f.dims['gru']
    nHru = f.dims['hru']

    # identify GRU or HRU dimension variables 
    gru_params = []
    hru_params = []
    for iParam in range(Nparam):
        iParam_name = object_parameters[iParam]
        iParam_dims = f[iParam_name].dims 

        if 'gru' in iParam_dims:
            gru_params.append(iParam_name)
        elif 'hru' in iParam_dims:
            hru_params.append(iParam_name)
    
    nGru_params = len(gru_params)
    nHru_params = len(hru_params)
    
    # 4. Create nGRU and nHru text files to store parameter sample values.
    for iGru in range(nGru):
        ofile = os.path.join(output_path,'GRU%d_param_samples.txt'%(iGru+1)) #eg, 'GRU1_param_samples.txt'
        with open(ofile, 'w') as f:
            f.write('Run\tobj.function\t')
            f.write('\t'.join(gru_params))
            f.write('\n')
    for iHru in range(nHru):
        ofile = os.path.join(output_path,'HRU%d_param_samples.txt'%(iHru+1)) #eg, 'GRU1_param_samples.txt'
        with open(ofile, 'w') as f:
            f.write('Run\tobj.function\t')
            f.write('\t'.join(hru_params))
            f.write('\n')        
            
    # 5. Loop sucess runs to read and store each run param values
    pbar = tqdm(total=Nsample)
    for iRun in range(Nsample):
#     for iRun in range(3):

        iRun_path = success_runs[iRun]
        paramFile = os.path.join(iRun_path,paramFile_basename)
        statsFile = os.path.join(iRun_path,statsFile_basename)        
        
        f = xr.open_dataset(paramFile)
        obj = np.loadtxt(statsFile, delimiter='\t', usecols=[0])
        obj = obj[0]*(-1) # negative KGE as the objective function of optimization

        # ------------ GRU param -----------------------
        # read all GRU param values
        gru_param_values = np.zeros((nGru,nGru_params))
        for iGru_param in range(nGru_params):
            iGru_param_name = gru_params[iGru_param] 
            gru_param_values[:,iGru_param] = f[iGru_param_name].values 

        # save GRU param values for each GRU individually
        for iGru in range(nGru):
            ofile = os.path.join(output_path,'GRU%d_param_samples.txt'%(iGru+1)) #eg, 'GRU1_param_samples.txt'
            with open(ofile, 'a') as fout:
                fout.write(('%d\t%.6E\t')%(iRun+1,obj))
                for iGru_param in range(nGru_params):
                    fout.write('%.6E\t'%(gru_param_values[iGru,iGru_param]))
                fout.write('\n')
        
        # ------------ HRU param -----------------------
        # read all HRU param values
        hru_param_values = np.zeros((nHru,nHru_params))
        for iHru_param in range(nHru_params):
            iHru_param_name = hru_params[iHru_param] 
            hru_param_values[:,iHru_param] = f[iHru_param_name].values 

        # save HRU param values for each HRU individually
        for iHru in range(nHru):
            ofile = os.path.join(output_path,'HRU%d_param_samples.txt'%(iHru+1)) #eg, 'GRU1_param_samples.txt'
            with open(ofile, 'a') as fout:
                fout.write(('%d\t%.6E\t')%(iRun+1,obj))
                for iHru_param in range(nHru_params):
                    fout.write('%.6E\t'%(hru_param_values[iHru,iHru_param]))
                fout.write('\n')                                           

        pbar.update(1)
    pbar.close()  
        


100%|██████████| 16929/16929 [36:09<00:00,  7.80it/s]  


In [46]:
print(("%.6E\t"*nGru_params)%(list(gru_param_values[iGru,:])))

TypeError: must be real number, not list

In [47]:
list(gru_param_values[iGru,:])

[13032.325247823033, 2.09648525]

In [44]:
hru_params

['k_macropore',
 'k_soil',
 'theta_sat',
 'aquiferBaseflowExp',
 'aquiferBaseflowRate',
 'qSurfScale',
 'summerLAI',
 'frozenPrecipMultip',
 'heightCanopyTop',
 'Fcapil']

In [25]:
obj

0.297968